# Timer Basic

This program was an introduction to the timer the system timer peripheral on RP2040. It provides a global microsecond timebase for the system, and generates interrupts based on this timebase. This programs fetches and prints the absolute time (the time elapsed since boot) on the serial monitor. The resources for the project include the [C SDK User Guide](https://datasheets.raspberrypi.org/pico/raspberry-pi-pico-c-sdk.pdf), the [RP2040 Datasheet](https://datasheets.raspberrypi.org/rp2040/rp2040-datasheet.pdf) and Prof. Hunter's [website](https://vha3.github.io/).

The timer peripheral on RP2040 supports:
- A single 64-bit counter, incrementing once per microsecond
- This counter can be read from a pair of latching registers, for race-free reads over a 32-bit bus.
- Four alarms: match on the lower 32 bits of counter, IRQ on match.

---

## The complete code

```c
/*
 * Parth Sarthi Sharma (pss242@cornell.edu)
 * Code based on examples from Raspberry Pi Foundation.
 * The code fetches the absolute time into a variable and prints out
 * the time on the serial monitor.
 */
#include <stdio.h> //The standard C library
#include "pico/stdlib.h" //Standard library for Pico
#include "pico/time.h" //The pico time library

int main(){
    stdio_init_all(); //Initialize all of the present standard stdio types that are linked into the binary
    while(1){ //While eternity
		uint64_t time = get_absolute_time(); //Get a representation of the current time
		printf("Time: %lld\n", time); //Print out the time on the serial monitor
		sleep_ms(1000); //Sleep for 1000 milliseconds
	}
}
```

---

<br>

## Stepping through the code

### Includes

The first lines of code in the C source file include some header files. One of these is standard C headers (`stdio.h`) and the others are headers which come from the C SDK for the Raspberry Pi Pico. The first of these, `pico/stdlib.h` is what the SDK calls a "High-Level API." These high-level API's "provide higher level functionality that isn’t hardware related or provides a richer set of functionality above the basic hardware interfaces." The architecture of this SDK is described at length in the SDK manual. All libraries within the SDK are INTERFACE libraries.

The next include pulls in hardware API which is not already brought in by `pico/stdlib.h`. This is the include `pico/time.h`. As the name suggestes, this interface library gives us access to the API associated with the pico time on the RP2040.

**Don't forget to link these in the CMakeLists.txt file!**

```c
#include <stdio.h>
#include "pico/stdlib.h"
#include "pico/time.h"
```

<br>

### The main function

#### Initializing communication

The first line in `main()` is a call to `stdio_init_all()`. This function initializes stdio to communicate through either UART or USB, depending on the configurations in the CMakeLists.txt file.

```c
stdio_init_all();
```

<br>

#### The infinite while loop

For this code, the infinite while loop follows the following algorithm over and over again:
- Fetch the absolute time.
- Print the fetched absolute time on the screen.
- Sleep for 1000 milliseconds.

In order to fetch the absolute time, I used the `get_absolute_time()` function. This function returns the time since boot in microseconds as an unsigned 64-bit integer. Therefore, it is going to continue to run for **5851444** years after it boots up! Once the value is fetched, it is printed out on the serial monitor and the CPU sleeps for 1 second.

```c
uint64_t time = get_absolute_time();
printf("Time: %lld\n", time);
sleep_ms(1000);
```

---

<br>

## Output

The output of the program is shown below. From the image it is verified that the time since boot (in microseconds) is being printed out at an interval of 1000 milliseconds.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/Pico/Files/TimerBasicOutput.png" style="width: 275px; height: 207px;" >
</div>
<figure>
    <center><figcaption>PWM signal generated based on value from the ADC</figcaption></center>
</figure>

---

## CMakeLists.txt

```cmake
cmake_minimum_required(VERSION 3.13)

include(pico_sdk_import.cmake)

project(TimerTest)

pico_sdk_init()

add_executable(TimerTest TimerTest.c)

pico_enable_stdio_usb(TimerTest 1)
pico_enable_stdio_uart(TimerTest 1)

pico_add_extra_outputs(TimerTest)

target_link_libraries(TimerTest pico_stdlib pico_time hardware_timer)
```

---